In [ ]:
output=True
frame["Cluster"]=clusters
Y=frame[str(predict)]
PFE=Y
frame.drop([predict],axis=1)
X_train=pd.DataFrame([])
X_test=pd.DataFrame([])
y_train=pd.DataFrame([])
y_test=pd.DataFrame([])
if output:
    Cluster_df=[]
for count,i in enumerate(sorted(frame["Cluster"].unique())):
    frame_t=frame[frame["Cluster"]==i]
    sampled_frame=frame_t.sample(int(sample/len(np.unique(clusters))),random_state=seed,replace=True)
    Y=PFE.iloc[sampled_frame.index.values]
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(sampled_frame, Y, test_size=0.2, random_state=seed)
    X_train=X_train.append(X_train_1)
    X_test=X_test.append(X_test_1)
    y_train=pd.concat([y_train,y_train_1.transpose()])
    y_test=pd.concat([y_test,y_test_1.transpose()])
    if output:
        Cluster_df.append(frame)
y_train=y_train.rename(columns={0:predict})
y_test=y_test.rename(columns={0:predict})

In [52]:
# help function
from transfer_learning import NeuralNet
from dataset_loader import data_loader, all_filter, get_descriptors, one_filter, data_scaler

# modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import os, sys
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

from tqdm import tqdm
from scipy.stats import pearsonr

import matplotlib.pyplot as plt 

# file name and data path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_path = os.getcwd()
file_name = 'data/CrystGrowthDesign_SI.csv'

"""
Data description.

    Descriptors:
        'void fraction', 'Vol. S.A.', 'Grav. S.A.', 'Pore diameter Limiting', 'Pore diameter Largest'
    Source task:
        'H2@100 bar/243K (wt%)'
    Target tasks:
        'H2@100 bar/130K (wt%)' 'CH4@100 bar/298 K (mg/g)' '5 bar Xe mol/kg' '5 bar Kr mol/kg'
"""

descriptor_columns = ['void fraction', 'Vol. S.A.', 'Grav. S.A.', 'Pore diameter Limiting', 'Pore diameter Largest']
one_filter_columns = ['H2@100 bar/243K (wt%)'] 
another_filter_columns = ['H2@100 bar/130K (wt%)'] 

data = data_loader(base_path, file_name)
data=data.reset_index(drop=True)
frame=data

In [53]:
from Statistics_helper import *
feat_5=frame[descriptor_columns]
r_feat_5=rescale(feat_5)
z=r_feat_5.copy()
print(r_feat_5.isnull().sum())
pc1,pc2,color=make_pca_agg_fit(1,r_feat_5,.1,5,func_give=KMeans,array_out=True,loud=False)
r_feat_5[one_filter_columns]=frame[one_filter_columns]

void fraction             0
Vol. S.A.                 0
Grav. S.A.                0
Pore diameter Limiting    0
Pore diameter Largest     0
dtype: int64


In [54]:
r_feat_5.isnull().sum()

void fraction             0
Vol. S.A.                 0
Grav. S.A.                0
Pore diameter Limiting    0
Pore diameter Largest     0
H2@100 bar/243K (wt%)     0
dtype: int64

In [49]:
z[1700:1709]

,void fraction,Vol. S.A.,Grav. S.A.,Pore diameter Limiting,Pore diameter Largest
1700,0.327648,0.521018,1.083468,-0.481335,-0.801966
1701,0.409872,0.320241,1.321452,-0.471908,-0.770705
1702,0.409872,0.125871,0.838315,-0.387068,-0.590956
1703,0.409872,0.057521,0.565924,-0.311655,-0.559695
1704,0.492097,-0.158208,1.268885,-0.264522,-0.504988
1705,-0.247920,1.541994,-0.008488,-0.858400,-1.145835
1706,0.492097,-0.045004,1.269363,-0.264522,-0.489358
1707,0.574321,-0.348306,0.822545,-0.160829,-0.325238
1708,0.574321,-0.478598,0.962564,-0.075990,-0.286162


In [25]:
r_feat_5[r_feat_5["H2@100 bar/243K (wt%)"].isnull()]

,void fraction,Vol. S.A.,Grav. S.A.,Pore diameter Limiting,Pore diameter Largest,H2@100 bar/243K (wt%)
1707,0.574321,-0.348306,0.822545,-0.160829,-0.325238,NaN
8181,0.738769,-1.373553,1.227787,1.036353,2.753954,NaN
13461,-0.494593,0.807233,-1.042141,-0.396495,-0.747260,NaN


In [51]:
(frame[one_filter_columns])[1700:1709]

,H2@100 bar/243K (wt%)
1700,5.0
1701,5.5
1702,5.4
1703,5.0
1704,6.4
1705,2.8
1706,6.3
1708,6.3
1709,6.9


In [33]:
data.iloc[1700:1708]

,MOF ID,void fraction,Vol. S.A.,Grav. S.A.,Pore diameter Limiting,Pore diameter Largest,H2@100 bar/77K (g/L),H2@100 bar/77K (mol/kg),H2@100 bar/77K (wt%),H2@100 bar/130K (g/L),...,5 bar Kr mol/kg,5 bar Selectivity,topology,First nodular symmetry code,First nodular character,First nodular ID,Second nodular symmetry code,Second nodular character,Second nodular ID,Connecting building block ID
1700,1702,0.92,1474,7733.0,14.1,16.4,50.4,132.3,20.9,24.5,...,2.6,2.1,ctn,3,organic,0,4,metallic,1,15.0
1701,1703,0.93,1380,8231.0,14.2,16.8,48.7,144.9,22.5,23.2,...,2.3,1.7,ctn,3,organic,0,4,metallic,1,16.0
1702,1704,0.93,1289,7220.0,15.1,19.1,49.1,137.5,21.6,24.2,...,2.9,2.2,ctn,3,organic,0,4,metallic,1,17.0
1703,1705,0.93,1257,6650.0,15.9,19.5,48.1,127.2,20.3,23.5,...,2.4,2.0,ctn,3,organic,0,4,metallic,1,18.0
1704,1706,0.94,1156,8121.0,16.4,20.2,47.0,164.8,24.8,22.3,...,2.7,1.7,ctn,3,organic,0,4,metallic,1,19.0
1705,1707,0.85,1952,5448.0,10.1,12.0,54.0,75.3,13.1,27.9,...,2.1,2.6,ctn,3,organic,0,4,metallic,1,1.0
1706,1708,0.94,1209,8122.0,16.4,20.4,47.6,159.7,24.2,23.0,...,2.8,1.9,ctn,3,organic,0,4,metallic,1,20.0
1708,1710,0.95,1067,7187.0,17.5,22.5,46.7,157.1,23.9,23.3,...,3.4,2.2,ctn,3,organic,0,4,metallic,1,22.0


In [36]:
data.reset_index(drop=True)[1700:1710]

,MOF ID,void fraction,Vol. S.A.,Grav. S.A.,Pore diameter Limiting,Pore diameter Largest,H2@100 bar/77K (g/L),H2@100 bar/77K (mol/kg),H2@100 bar/77K (wt%),H2@100 bar/130K (g/L),...,5 bar Kr mol/kg,5 bar Selectivity,topology,First nodular symmetry code,First nodular character,First nodular ID,Second nodular symmetry code,Second nodular character,Second nodular ID,Connecting building block ID
1700,1702,0.92,1474,7733.0,14.1,16.4,50.4,132.3,20.9,24.5,...,2.6,2.1,ctn,3,organic,0,4,metallic,1,15.0
1701,1703,0.93,1380,8231.0,14.2,16.8,48.7,144.9,22.5,23.2,...,2.3,1.7,ctn,3,organic,0,4,metallic,1,16.0
1702,1704,0.93,1289,7220.0,15.1,19.1,49.1,137.5,21.6,24.2,...,2.9,2.2,ctn,3,organic,0,4,metallic,1,17.0
1703,1705,0.93,1257,6650.0,15.9,19.5,48.1,127.2,20.3,23.5,...,2.4,2.0,ctn,3,organic,0,4,metallic,1,18.0
1704,1706,0.94,1156,8121.0,16.4,20.2,47.0,164.8,24.8,22.3,...,2.7,1.7,ctn,3,organic,0,4,metallic,1,19.0
1705,1707,0.85,1952,5448.0,10.1,12.0,54.0,75.3,13.1,27.9,...,2.1,2.6,ctn,3,organic,0,4,metallic,1,1.0
1706,1708,0.94,1209,8122.0,16.4,20.4,47.6,159.7,24.2,23.0,...,2.8,1.9,ctn,3,organic,0,4,metallic,1,20.0
1707,1710,0.95,1067,7187.0,17.5,22.5,46.7,157.1,23.9,23.3,...,3.4,2.2,ctn,3,organic,0,4,metallic,1,22.0
1708,1711,0.95,1006,7480.0,18.4,23.0,45.8,170.1,25.4,22.7,...,3.3,2.1,ctn,3,organic,0,4,metallic,1,23.0
1709,1712,0.95,975,7912.0,19.1,23.9,44.9,181.9,26.7,22.0,...,3.2,1.9,ctn,3,organic,0,4,metallic,1,24.0
